In [1]:
#importing packages
import pandas as pd
import numpy as np
import os
import datetime
import sklearn as sk
import random

from sklearn import preprocessing

#imporintg tsfresh
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

/sw/acf/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#reading the data
ds = pd.read_csv("tempStudy_deIDNew.csv")

In [3]:
#making a new dataframe
df = pd.DataFrame(ds)

In [4]:
#[testing purposes]

#grouping the  data by IDs
grouped_data = df.groupby(['ID']).mean()
#grouped_data.to_csv("groupby_mean.csv",index=False)
grouped_data.head()

,MDC_CO2_RESP_RATE,MDC_CONC_AWAY_CO2_ET,MDC_CONC_AWAY_CO2_INSP,MDC_ECG_AMPL_ST_AVF,MDC_ECG_AMPL_ST_AVL,MDC_ECG_AMPL_ST_AVR,MDC_ECG_AMPL_ST_I,MDC_ECG_AMPL_ST_II,MDC_ECG_AMPL_ST_III,MDC_ECG_AMPL_ST_V,...,MDC_PRESS_BLD_VEN_CENT_MEAN,MDC_PRESS_CEREB_PERF,MDC_PRESS_CUFF_DIA,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_SYS,MDC_PRESS_INTRA_CRAN_MEAN,MDC_PULS_OXIM_PULS_RATE,MDC_PULS_OXIM_SAT_O2,MDC_TEMP,MDC_TTHOR_RESP_RATE
ID,,,,,,,,,,,,,,,,,,,,,
2266291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.863636,62.636364,91.045455,NaN,147.277078,97.359362,NaN,44.659646
2330393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,59.012456,77.556303,101.709964,NaN,123.738084,98.008688,NaN,53.072094
2330507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,54.278571,73.564626,99.557143,NaN,127.028986,98.734533,NaN,39.591433
2336509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,64.200000,82.000000,111.000000,NaN,127.519731,94.734677,NaN,39.542205
2350749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,55.021401,71.379048,97.955253,NaN,128.861942,93.020329,NaN,50.580360


In [5]:
#[testing purposes]

#finding, sorting and exporting the Non-Nan values in columns
rows_nonNan = grouped_data.count(axis = 'rows')
per_nonNan_columns = (rows_nonNan/722)*100
per_nonNan_columns_sorted = per_nonNan_columns.sort_values(ascending = False)
#per_nonNan_columns_sorted.to_csv("groupByFIN_FeaturePercent.csv")
per_nonNan_columns_sorted

MDC_PRESS_CUFF_SYS             100.000000
MDC_PRESS_CUFF_MEAN            100.000000
MDC_PRESS_CUFF_DIA             100.000000
MDC_PULS_OXIM_SAT_O2            99.861496
MDC_PULS_OXIM_PULS_RATE         99.861496
MDC_ECG_CARD_BEAT_RATE          99.168975
MDC_TTHOR_RESP_RATE             97.922438
MDC_PRESS_BLD_ART_SYS           25.900277
MDC_PRESS_BLD_ART_DIA           25.900277
MDC_PRESS_BLD_ART_MEAN          25.900277
MDC_ECG_V_P_C_RATE              25.484765
MDC_TEMP                        18.005540
MDC_ECG_AMPL_ST_II              16.897507
MDC_PRESS_BLD_VEN_CENT_MEAN      6.371191
MDC_ECG_AMPL_ST_I                5.401662
MDC_ECG_AMPL_ST_III              4.432133
MDC_PRESS_INTRA_CRAN_MEAN        2.908587
MDC_PRESS_BLD_MEAN               2.770083
MDC_PRESS_BLD_SYS                2.770083
MDC_PRESS_BLD_DIA                2.770083
MDC_PRESS_CEREB_PERF             1.523546
MDC_ECG_PACED_BEAT_RATE          0.692521
MDC_CONC_AWAY_CO2_ET             0.415512
MDC_CONC_AWAY_CO2_INSP           0

In [6]:
#making the subset of the original dataframe df

#create a var of all of these names and can do a subset of dataframe
subset_columns = ['ID','Date','MDC_PRESS_CUFF_SYS','MDC_PRESS_CUFF_MEAN','MDC_PRESS_CUFF_DIA','MDC_PULS_OXIM_SAT_O2','MDC_PULS_OXIM_PULS_RATE','MDC_ECG_CARD_BEAT_RATE','MDC_TTHOR_RESP_RATE', 'MDC_TEMP']

df1 = df[subset_columns]
df1

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP
0,2330393,2018-01-29 09:16:03,85.0,73.0,61.0,NaN,NaN,NaN,NaN,NaN
1,2330393,2018-01-29 12:26:31,NaN,NaN,NaN,99.0,129.0,125.0,50.0,NaN
2,2330393,2018-01-29 12:27:31,NaN,NaN,NaN,100.0,130.0,128.0,NaN,NaN
3,2330393,2018-01-29 12:28:31,NaN,NaN,NaN,99.0,126.0,125.0,NaN,NaN
4,2330393,2018-01-29 12:29:31,NaN,NaN,NaN,99.0,123.0,122.0,NaN,NaN
5,2330393,2018-01-29 12:30:31,NaN,NaN,NaN,98.0,126.0,126.0,NaN,NaN
6,2330393,2018-01-29 12:31:31,NaN,NaN,NaN,99.0,119.0,117.0,NaN,NaN
7,2330393,2018-01-29 12:32:31,NaN,NaN,NaN,99.0,126.0,124.0,NaN,NaN
8,2330393,2018-01-29 12:33:31,NaN,NaN,NaN,99.0,121.0,120.0,NaN,NaN
9,2330393,2018-01-29 12:34:31,NaN,NaN,NaN,97.0,133.0,131.0,NaN,NaN


In [7]:
df1.dtypes
#df1.head()
#df.describe()

ID                           int64
Date                        object
MDC_PRESS_CUFF_SYS         float64
MDC_PRESS_CUFF_MEAN        float64
MDC_PRESS_CUFF_DIA         float64
MDC_PULS_OXIM_SAT_O2       float64
MDC_PULS_OXIM_PULS_RATE    float64
MDC_ECG_CARD_BEAT_RATE     float64
MDC_TTHOR_RESP_RATE        float64
MDC_TEMP                   float64
dtype: object

In [8]:
#looking at the temperature greater than equal to 38
#tempEp= df1[df1.MDC_TEMP >= 38]
#tempEp

df1['fever'] = np.where(df1['MDC_TEMP'] >= 38, 1, 0)
df2 = df1.groupby(['ID']).sum()
df2
df3 = df2.sort_values(ascending = False, by = 'fever')


#df3.to_csv("df3_test.csv",index=False)


#for each id, could check for the time that satisfies this condition >=38
#also give a count of fever episodes (>=38) for wach id

/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [9]:
#[testing purposes]
#count for 38 temp
df3[df3["fever"]>=1].count()["fever"]

76

In [10]:
#[testing purposes]
#counting the unique values in the columns
#unId = df1["ID"].unique()
#list(df1["ID"]).count(74962868)

countID = df1.nunique()
countID

ID                             722
Date                       2948461
MDC_PRESS_CUFF_SYS             179
MDC_PRESS_CUFF_MEAN            166
MDC_PRESS_CUFF_DIA             148
MDC_PULS_OXIM_SAT_O2           100
MDC_PULS_OXIM_PULS_RATE        213
MDC_ECG_CARD_BEAT_RATE         283
MDC_TTHOR_RESP_RATE            147
MDC_TEMP                       360
fever                            2
dtype: int64

In [11]:
#[testing purposes]
#finding unique dates and time for unique IDs

groupedId = df1.groupby('ID')
dfTest = groupedId.apply(lambda x: x['Date'].unique())
dfTest

ID
2266291     [2018-05-12 07:00:44, 2018-05-12 07:12:05, 201...
2330393     [2018-01-29 09:16:03, 2018-01-29 12:26:31, 201...
2330507     [2018-01-29 12:18:21, 2018-01-29 12:26:42, 201...
2336509     [2018-04-28 20:12:27, 2018-04-28 20:13:25, 201...
2350749     [2018-01-29 12:14:51, 2018-01-29 12:26:32, 201...
2369063     [2018-06-27 20:50:31, 2018-06-27 20:51:31, 201...
2371171     [2018-01-29 09:47:58, 2018-01-29 12:26:36, 201...
2373966     [2018-03-13 21:04:30, 2018-03-13 21:05:30, 201...
2385103     [2018-06-18 20:25:48, 2018-06-18 20:25:59, 201...
2387383     [2018-03-01 17:10:27, 2018-03-01 17:11:27, 201...
2396401     [2018-01-29 09:59:23, 2018-01-29 12:26:36, 201...
2398315     [2018-01-29 11:00:32, 2018-01-29 12:26:37, 201...
2398642     [2018-02-04 16:00:35, 2018-02-04 16:06:50, 201...
2398991     [2018-02-09 10:30:29, 2018-02-09 10:33:21, 201...
2399089     [2018-01-31 00:51:12, 2018-01-31 00:52:12, 201...
2400740     [2018-01-30 14:00:24, 2018-01-30 14:04:01, 201...
24017

In [12]:
#converting the Date to datetime
df1['Date'] = pd.to_datetime(df1['Date'],infer_datetime_format=True)
df1['Date_timestamp'] = df1['Date'].astype('int64')/10**9
#df1['Date_timestamp'] = df1['Date']
df1



/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp
0,2330393,2018-01-29 09:16:03,85.0,73.0,61.0,NaN,NaN,NaN,NaN,NaN,0,1.517217e+09
1,2330393,2018-01-29 12:26:31,NaN,NaN,NaN,99.0,129.0,125.0,50.0,NaN,0,1.517229e+09
2,2330393,2018-01-29 12:27:31,NaN,NaN,NaN,100.0,130.0,128.0,NaN,NaN,0,1.517229e+09
3,2330393,2018-01-29 12:28:31,NaN,NaN,NaN,99.0,126.0,125.0,NaN,NaN,0,1.517229e+09
4,2330393,2018-01-29 12:29:31,NaN,NaN,NaN,99.0,123.0,122.0,NaN,NaN,0,1.517229e+09
5,2330393,2018-01-29 12:30:31,NaN,NaN,NaN,98.0,126.0,126.0,NaN,NaN,0,1.517229e+09
6,2330393,2018-01-29 12:31:31,NaN,NaN,NaN,99.0,119.0,117.0,NaN,NaN,0,1.517229e+09
7,2330393,2018-01-29 12:32:31,NaN,NaN,NaN,99.0,126.0,124.0,NaN,NaN,0,1.517229e+09
8,2330393,2018-01-29 12:33:31,NaN,NaN,NaN,99.0,121.0,120.0,NaN,NaN,0,1.517229e+09
9,2330393,2018-01-29 12:34:31,NaN,NaN,NaN,97.0,133.0,131.0,NaN,NaN,0,1.517229e+09


In [13]:
dfxx = df1
#del df1

In [14]:
dfxx['Date_timestamp'].values

array([  1.51721736e+09,   1.51722879e+09,   1.51722885e+09, ...,
         1.53015297e+09,   1.53015303e+09,   1.53015309e+09])

In [15]:
dfxx

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp
0,2330393,2018-01-29 09:16:03,85.0,73.0,61.0,NaN,NaN,NaN,NaN,NaN,0,1.517217e+09
1,2330393,2018-01-29 12:26:31,NaN,NaN,NaN,99.0,129.0,125.0,50.0,NaN,0,1.517229e+09
2,2330393,2018-01-29 12:27:31,NaN,NaN,NaN,100.0,130.0,128.0,NaN,NaN,0,1.517229e+09
3,2330393,2018-01-29 12:28:31,NaN,NaN,NaN,99.0,126.0,125.0,NaN,NaN,0,1.517229e+09
4,2330393,2018-01-29 12:29:31,NaN,NaN,NaN,99.0,123.0,122.0,NaN,NaN,0,1.517229e+09
5,2330393,2018-01-29 12:30:31,NaN,NaN,NaN,98.0,126.0,126.0,NaN,NaN,0,1.517229e+09
6,2330393,2018-01-29 12:31:31,NaN,NaN,NaN,99.0,119.0,117.0,NaN,NaN,0,1.517229e+09
7,2330393,2018-01-29 12:32:31,NaN,NaN,NaN,99.0,126.0,124.0,NaN,NaN,0,1.517229e+09
8,2330393,2018-01-29 12:33:31,NaN,NaN,NaN,99.0,121.0,120.0,NaN,NaN,0,1.517229e+09
9,2330393,2018-01-29 12:34:31,NaN,NaN,NaN,97.0,133.0,131.0,NaN,NaN,0,1.517229e+09


In [16]:
#setting condiiton for febrile temperature, calulating the delta and chceking to meet criteria

#convert the epoch time to datetine?????????

dfxx['fever_date'] = np.where(dfxx['MDC_TEMP'] >= 38, dfxx['Date'], None)
dfxx['fever_date'] = pd.to_datetime(dfxx['fever_date'],infer_datetime_format=True)

dfxx['fever_time'] = np.where(dfxx['MDC_TEMP'] >= 38, dfxx['Date_timestamp'], None)
#dfxx['fever_time'] = pd.to_datetime(dfxx['fever_time'],infer_datetime_format=True)
#dfxx = dfxx.drop('level_0', axis=1)
dfxx = dfxx.sort_values(by=['ID', 'Date_timestamp'], ascending=[True,True]).groupby(['ID'], as_index = False).fillna(method='bfill').reset_index(drop = True)
dfxx['febrile_time'] = dfxx['fever_time']

#df['TradeDate'] = pd.to_datetime(df['TradeDate'], format='%Y%m%d.0')
#dfxx['febrile_date'] = pd.to_datetime(dfxx['febrile_date'],infer_datetime_format=True)

#dfxx['delta'] = ((dfxx['febrile_date'] - dfxx['Date']).dt.seconds)
#dfxx['delta'] = ((dfxx['febrile_date'] - dfxx['Date_timestamp']).dt.seconds)
dfxx['delta'] = ((dfxx['febrile_time'] - dfxx['Date_timestamp']))

#dfxx['febrile_date'] = pd.to_datetime(dfxx['febrile_time'],infer_datetime_format=True)

dfxx['metCriteria'] = np.where(((dfxx['delta'] >0) & (dfxx['delta'] < 21600)), 1, 0)


/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarni

In [17]:
dfxx

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_date,fever_time,febrile_time,delta,metCriteria
0,2266291,2018-05-12 07:00:44,82.0,62.0,40.0,100.0,165.0,164.0,81.0,NaN,0,1.526108e+09,NaT,NaN,NaN,NaN,0
1,2266291,2018-05-12 07:12:05,112.0,81.0,51.0,100.0,165.0,164.0,81.0,NaN,0,1.526109e+09,NaT,NaN,NaN,NaN,0
2,2266291,2018-05-12 07:13:05,112.0,81.0,51.0,100.0,163.0,163.0,73.0,NaN,0,1.526109e+09,NaT,NaN,NaN,NaN,0
3,2266291,2018-05-12 07:14:05,112.0,81.0,51.0,100.0,163.0,162.0,56.0,NaN,0,1.526109e+09,NaT,NaN,NaN,NaN,0
4,2266291,2018-05-12 07:15:05,112.0,81.0,51.0,100.0,163.0,163.0,85.0,NaN,0,1.526109e+09,NaT,NaN,NaN,NaN,0
5,2266291,2018-05-12 07:15:39,112.0,81.0,51.0,100.0,167.0,166.0,83.0,NaN,0,1.526109e+09,NaT,NaN,NaN,NaN,0
6,2266291,2018-05-12 07:16:05,100.0,54.0,33.0,100.0,167.0,166.0,83.0,NaN,0,1.526109e+09,NaT,NaN,NaN,NaN,0
7,2266291,2018-05-12 07:17:05,100.0,54.0,33.0,100.0,165.0,164.0,62.0,NaN,0,1.526109e+09,NaT,NaN,NaN,NaN,0
8,2266291,2018-05-12 07:18:05,100.0,54.0,33.0,100.0,163.0,163.0,88.0,NaN,0,1.526109e+09,NaT,NaN,NaN,NaN,0
9,2266291,2018-05-12 07:19:05,100.0,54.0,33.0,100.0,164.0,163.0,99.0,NaN,0,1.526110e+09,NaT,NaN,NaN,NaN,0


In [18]:
dfSub = dfxx
dfsub2 = dfSub[dfSub['MDC_TEMP'].notnull()]
dfsub2['delta'] = dfsub2['delta'].fillna(50000)

/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
#setting up sample_id for ts_FRESH

startHour = np.arange(0,6) ## Hours before alert time
endObservation = 1 ## Observational Window length of each patient in hours
lenStart = len(startHour)

for i in range(0,lenStart):
    #print("Test")
    dfsub2.loc[(dfsub2['delta'] > startHour[i] * 3600) & (dfsub2['delta'] <=(startHour[i]+endObservation) * 3600), 'sample_id'] = dfsub2["ID"].astype(str) + "_" + str(i+1)
    

/sw/acf/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/sw/acf/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
#dfsub2.drop(["sample_id"], axis=1, inplace=True)

In [50]:
dfsub2.dropna(subset= ["fever_time"])
dfsub2

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_date,fever_time,febrile_time,delta,metCriteria,sample_id,hourAway,nFeverEventID
410638,2398315,2018-01-29 10:28:43,141.0,110.0,87.0,100.0,138.0,135.0,29.0,39.5,0,1.517222e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,139471.0,0,NaN,38,2398315_1517361194.0
410639,2398315,2018-01-29 11:00:32,95.0,74.0,59.0,100.0,138.0,135.0,29.0,39.5,0,1.517224e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,137562.0,0,NaN,38,2398315_1517361194.0
410640,2398315,2018-01-29 12:26:33,158.0,109.0,90.0,100.0,138.0,135.0,29.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132401.0,0,NaN,36,2398315_1517361194.0
410641,2398315,2018-01-29 12:26:37,158.0,109.0,90.0,94.0,107.0,106.0,29.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132397.0,0,NaN,36,2398315_1517361194.0
410642,2398315,2018-01-29 12:27:33,158.0,109.0,90.0,91.0,147.0,137.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132341.0,0,NaN,36,2398315_1517361194.0
410643,2398315,2018-01-29 12:27:37,158.0,109.0,90.0,93.0,106.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132337.0,0,NaN,36,2398315_1517361194.0
410644,2398315,2018-01-29 12:28:33,158.0,109.0,90.0,100.0,133.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132281.0,0,NaN,36,2398315_1517361194.0
410645,2398315,2018-01-29 12:28:37,158.0,109.0,90.0,94.0,106.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132277.0,0,NaN,36,2398315_1517361194.0
410646,2398315,2018-01-29 12:29:19,158.0,109.0,90.0,96.0,139.0,106.0,28.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132235.0,0,NaN,36,2398315_1517361194.0
410647,2398315,2018-01-29 12:29:33,97.0,74.0,58.0,96.0,139.0,106.0,28.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132221.0,0,NaN,36,2398315_1517361194.0


In [21]:
#### RK edits
#dfsub2 = dfsub2[dfsub2['metCriteria'] == 1]
dfsub2['hourAway'] = (dfsub2['delta'] / 3600).astype(int)
dfsub2['nFeverEventID'] = dfsub2['ID'].astype(str) + '_' + dfsub2['fever_time'].astype(str)

/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [46]:
dfsub2

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_date,fever_time,febrile_time,delta,metCriteria,sample_id,hourAway,nFeverEventID
410638,2398315,2018-01-29 10:28:43,141.0,110.0,87.0,100.0,138.0,135.0,29.0,39.5,0,1.517222e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,139471.0,0,NaN,38,2398315_1517361194.0
410639,2398315,2018-01-29 11:00:32,95.0,74.0,59.0,100.0,138.0,135.0,29.0,39.5,0,1.517224e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,137562.0,0,NaN,38,2398315_1517361194.0
410640,2398315,2018-01-29 12:26:33,158.0,109.0,90.0,100.0,138.0,135.0,29.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132401.0,0,NaN,36,2398315_1517361194.0
410641,2398315,2018-01-29 12:26:37,158.0,109.0,90.0,94.0,107.0,106.0,29.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132397.0,0,NaN,36,2398315_1517361194.0
410642,2398315,2018-01-29 12:27:33,158.0,109.0,90.0,91.0,147.0,137.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132341.0,0,NaN,36,2398315_1517361194.0
410643,2398315,2018-01-29 12:27:37,158.0,109.0,90.0,93.0,106.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132337.0,0,NaN,36,2398315_1517361194.0
410644,2398315,2018-01-29 12:28:33,158.0,109.0,90.0,100.0,133.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132281.0,0,NaN,36,2398315_1517361194.0
410645,2398315,2018-01-29 12:28:37,158.0,109.0,90.0,94.0,106.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132277.0,0,NaN,36,2398315_1517361194.0
410646,2398315,2018-01-29 12:29:19,158.0,109.0,90.0,96.0,139.0,106.0,28.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132235.0,0,NaN,36,2398315_1517361194.0
410647,2398315,2018-01-29 12:29:33,97.0,74.0,58.0,96.0,139.0,106.0,28.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,132221.0,0,NaN,36,2398315_1517361194.0


In [23]:
####

#1. for those with fever: filter out delta's that are above 6 hours or below 0, save to a dataframe X1
#2. for those who never had fever (feverTime = nan): create a random feverTime, generate delta and join to the previous dataframe
#3. pass this through tsfresh

###

#dfsub2[(dfsub2["delta"] >= 21600)]

In [26]:
###1. for those with fever: filter out delta's that are above 6 hours or below 0, save to a dataframe X1
x1 = dfsub2

#x1["delta"] = np.where(((x1["delta"] >= 21600) or (x1["delta"] <= 0)),np.NaN , x1["delta"] )
#x1 = x1["fever_time"].dropna()
#x1 = x1.drop(["deltaNew"], axis = 1, inplace = True)

x1 = x1.dropna(subset = ["fever_time"])

def filter_delta(value):
    if value >= 21600 or value <= 0:
        return np.NAN
    return value
x1['delta'] = [filter_delta(i) for i in x1['delta']]


x1

/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_date,fever_time,febrile_time,delta,metCriteria,sample_id,hourAway,nFeverEventID
410638,2398315,2018-01-29 10:28:43,141.0,110.0,87.0,100.0,138.0,135.0,29.0,39.5,0,1.517222e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,38,2398315_1517361194.0
410639,2398315,2018-01-29 11:00:32,95.0,74.0,59.0,100.0,138.0,135.0,29.0,39.5,0,1.517224e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,38,2398315_1517361194.0
410640,2398315,2018-01-29 12:26:33,158.0,109.0,90.0,100.0,138.0,135.0,29.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410641,2398315,2018-01-29 12:26:37,158.0,109.0,90.0,94.0,107.0,106.0,29.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410642,2398315,2018-01-29 12:27:33,158.0,109.0,90.0,91.0,147.0,137.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410643,2398315,2018-01-29 12:27:37,158.0,109.0,90.0,93.0,106.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410644,2398315,2018-01-29 12:28:33,158.0,109.0,90.0,100.0,133.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410645,2398315,2018-01-29 12:28:37,158.0,109.0,90.0,94.0,106.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410646,2398315,2018-01-29 12:29:19,158.0,109.0,90.0,96.0,139.0,106.0,28.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410647,2398315,2018-01-29 12:29:33,97.0,74.0,58.0,96.0,139.0,106.0,28.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0


In [27]:
x1['fever_time'].describe()

count    4.934730e+05
mean     1.525415e+09
std      3.454650e+06
min      1.517236e+09
25%      1.523642e+09
50%      1.525937e+09
75%      1.527941e+09
max      1.530092e+09
Name: fever_time, dtype: float64

In [28]:
x1[(x1["delta"] >= 0)]

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_date,fever_time,febrile_time,delta,metCriteria,sample_id,hourAway,nFeverEventID
414399,2398315,2018-01-30 19:13:23,131.0,98.0,66.0,97.0,114.0,113.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21591.0,1,2398315_6,5,2398315_1517361194.0
414400,2398315,2018-01-30 19:14:06,131.0,98.0,66.0,100.0,109.0,109.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21548.0,1,2398315_6,5,2398315_1517361194.0
414401,2398315,2018-01-30 19:14:23,131.0,98.0,66.0,95.0,116.0,115.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21531.0,1,2398315_6,5,2398315_1517361194.0
414402,2398315,2018-01-30 19:15:06,131.0,98.0,66.0,100.0,107.0,107.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21488.0,1,2398315_6,5,2398315_1517361194.0
414403,2398315,2018-01-30 19:15:23,131.0,98.0,66.0,96.0,116.0,116.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21471.0,1,2398315_6,5,2398315_1517361194.0
414404,2398315,2018-01-30 19:16:06,131.0,98.0,66.0,100.0,121.0,121.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21428.0,1,2398315_6,5,2398315_1517361194.0
414405,2398315,2018-01-30 19:16:23,131.0,98.0,66.0,96.0,114.0,114.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21411.0,1,2398315_6,5,2398315_1517361194.0
414406,2398315,2018-01-30 19:17:06,131.0,98.0,66.0,100.0,122.0,122.0,23.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21368.0,1,2398315_6,5,2398315_1517361194.0
414407,2398315,2018-01-30 19:17:23,131.0,98.0,66.0,97.0,117.0,116.0,23.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21351.0,1,2398315_6,5,2398315_1517361194.0
414408,2398315,2018-01-30 19:18:06,131.0,98.0,66.0,100.0,110.0,110.0,22.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,21308.0,1,2398315_6,5,2398315_1517361194.0


In [30]:
###2. for those who never had fever (feverTime = nan): create a random feverTime, generate delta and join to the previous df
random.seed(10)

x2 = dfsub2

x2 = x2.loc[x2['fever_time'].isnull()]

x2['fever_time'] = [random.uniform(1.55,1.559)*(10**9) for i in range (x2.shape[0])]
#random number generated should be const.

#x2['fever_time_datetime'] = pd.to_datetime(x2['fever_time'],infer_datetime_format=True, unit = 's') #, unit = 's')

#x2['fever_time'] = np.where(x2['MDC_TEMP'] >= 25, x2['Date_timestamp'], None)
# x2['fever_time'] = pd.to_datetime(x2['fever_time'],infer_datetime_format=True)
# x2 = x2.drop('level_0', axis=1)
# x2 = x2.sort_values(by=['ID', 'Date_timestamp'], ascending=[True,True]).groupby(['ID'], as_index = False).fillna(method='bfill').reset_index(drop = True)

#x2['febrile_time'] = x2['fever_time']

#x2['delta'] = x2['fever_time'] - x2['Date_timestamp']

#x2.to_csv("x1_x2.csv")


/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [51]:
x2

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_date,fever_time,febrile_time,delta,metCriteria,sample_id,hourAway,nFeverEventID
437165,2398315,2018-02-10 03:45:40,115.0,85.0,65.0,94.0,90.0,90.0,22.0,37.9,0,1.518234e+09,NaT,1.555143e+09,NaN,3.690828e+07,0,NaN,13,2398315_nan
437166,2398315,2018-02-10 03:46:27,115.0,88.0,68.0,94.0,90.0,90.0,22.0,37.9,0,1.518234e+09,NaT,1.553860e+09,NaN,3.562561e+07,0,NaN,13,2398315_nan
437167,2398315,2018-02-10 03:46:28,115.0,88.0,68.0,95.0,132.0,132.0,22.0,37.9,0,1.518234e+09,NaT,1.555203e+09,NaN,3.696843e+07,0,NaN,13,2398315_nan
437168,2398315,2018-02-10 03:46:31,115.0,88.0,68.0,95.0,132.0,132.0,15.0,37.9,0,1.518234e+09,NaT,1.551855e+09,NaN,3.362049e+07,0,NaN,13,2398315_nan
437169,2398315,2018-02-10 03:47:27,115.0,88.0,68.0,94.0,91.0,91.0,23.0,37.9,0,1.518234e+09,NaT,1.557320e+09,NaN,3.908544e+07,0,NaN,13,2398315_nan
437170,2398315,2018-02-10 03:47:31,115.0,88.0,68.0,95.0,132.0,131.0,15.0,37.9,0,1.518234e+09,NaT,1.557412e+09,NaN,3.917785e+07,0,NaN,13,2398315_nan
437171,2398315,2018-02-10 03:48:27,115.0,88.0,68.0,95.0,94.0,95.0,22.0,37.9,0,1.518235e+09,NaT,1.555881e+09,NaN,3.764675e+07,0,NaN,13,2398315_nan
437172,2398315,2018-02-10 03:48:31,115.0,88.0,68.0,95.0,132.0,131.0,15.0,37.9,0,1.518235e+09,NaT,1.551442e+09,NaN,3.320756e+07,0,NaN,13,2398315_nan
437173,2398315,2018-02-10 03:49:27,115.0,88.0,68.0,95.0,89.0,88.0,19.0,37.9,0,1.518235e+09,NaT,1.554686e+09,NaN,3.645146e+07,0,NaN,13,2398315_nan
437174,2398315,2018-02-10 03:49:28,115.0,88.0,68.0,94.0,132.0,131.0,19.0,37.9,0,1.518235e+09,NaT,1.552950e+09,NaN,3.471539e+07,0,NaN,13,2398315_nan


In [32]:
x2['fever_time'].values

array([  1.55514262e+09,   1.55386000e+09,   1.55520282e+09, ...,
         1.55678642e+09,   1.55025707e+09,   1.55129447e+09])

In [33]:
x2['delta'] = x2['fever_time'] - x2['Date_timestamp']

#x2["delta"] = x2['fever_time'].subtract(['Date_timestamp'])

/nics/e/sw/cs400_centos7.3_acfsoftware/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
x2['delta'].describe()

count    5.478150e+05
mean     3.090198e+07
std      4.741869e+06
min      1.986111e+07
25%      2.724824e+07
50%      3.095911e+07
75%      3.450359e+07
max      4.175980e+07
Name: delta, dtype: float64

In [36]:
#combining two dataframes
frames = [x1, x2]
x3 = pd.concat(frames)

In [37]:
x3

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_date,fever_time,febrile_time,delta,metCriteria,sample_id,hourAway,nFeverEventID
410638,2398315,2018-01-29 10:28:43,141.0,110.0,87.0,100.0,138.0,135.0,29.0,39.5,0,1.517222e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,38,2398315_1517361194.0
410639,2398315,2018-01-29 11:00:32,95.0,74.0,59.0,100.0,138.0,135.0,29.0,39.5,0,1.517224e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,38,2398315_1517361194.0
410640,2398315,2018-01-29 12:26:33,158.0,109.0,90.0,100.0,138.0,135.0,29.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410641,2398315,2018-01-29 12:26:37,158.0,109.0,90.0,94.0,107.0,106.0,29.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410642,2398315,2018-01-29 12:27:33,158.0,109.0,90.0,91.0,147.0,137.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410643,2398315,2018-01-29 12:27:37,158.0,109.0,90.0,93.0,106.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410644,2398315,2018-01-29 12:28:33,158.0,109.0,90.0,100.0,133.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410645,2398315,2018-01-29 12:28:37,158.0,109.0,90.0,94.0,106.0,106.0,27.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410646,2398315,2018-01-29 12:29:19,158.0,109.0,90.0,96.0,139.0,106.0,28.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0
410647,2398315,2018-01-29 12:29:33,97.0,74.0,58.0,96.0,139.0,106.0,28.0,39.5,0,1.517229e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,NaN,0,NaN,36,2398315_1517361194.0


In [38]:
#removing nas from delta value
x3 = x3.dropna(subset = ["delta"])
x3

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_date,fever_time,febrile_time,delta,metCriteria,sample_id,hourAway,nFeverEventID
414399,2398315,2018-01-30 19:13:23,131.0,98.0,66.0,97.0,114.0,113.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.159100e+04,1,2398315_6,5,2398315_1517361194.0
414400,2398315,2018-01-30 19:14:06,131.0,98.0,66.0,100.0,109.0,109.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.154800e+04,1,2398315_6,5,2398315_1517361194.0
414401,2398315,2018-01-30 19:14:23,131.0,98.0,66.0,95.0,116.0,115.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.153100e+04,1,2398315_6,5,2398315_1517361194.0
414402,2398315,2018-01-30 19:15:06,131.0,98.0,66.0,100.0,107.0,107.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.148800e+04,1,2398315_6,5,2398315_1517361194.0
414403,2398315,2018-01-30 19:15:23,131.0,98.0,66.0,96.0,116.0,116.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.147100e+04,1,2398315_6,5,2398315_1517361194.0
414404,2398315,2018-01-30 19:16:06,131.0,98.0,66.0,100.0,121.0,121.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.142800e+04,1,2398315_6,5,2398315_1517361194.0
414405,2398315,2018-01-30 19:16:23,131.0,98.0,66.0,96.0,114.0,114.0,24.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.141100e+04,1,2398315_6,5,2398315_1517361194.0
414406,2398315,2018-01-30 19:17:06,131.0,98.0,66.0,100.0,122.0,122.0,23.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.136800e+04,1,2398315_6,5,2398315_1517361194.0
414407,2398315,2018-01-30 19:17:23,131.0,98.0,66.0,97.0,117.0,116.0,23.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.135100e+04,1,2398315_6,5,2398315_1517361194.0
414408,2398315,2018-01-30 19:18:06,131.0,98.0,66.0,100.0,110.0,110.0,22.0,39.5,0,1.517340e+09,2018-01-31 01:13:14,1.517361e+09,1.517361e+09,2.130800e+04,1,2398315_6,5,2398315_1517361194.0


In [39]:
x3.describe()

,ID,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_time,febrile_time,delta,metCriteria,hourAway
count,6.540960e+05,649689.000000,649732.000000,649689.000000,653950.000000,653951.000000,654077.000000,645700.000000,654096.000000,654096.0,6.540960e+05,6.540960e+05,1.062810e+05,6.540960e+05,654096.000000,654096.000000
mean,4.225649e+07,103.642016,74.951715,57.209337,94.326982,121.658912,121.254838,30.793799,35.998616,0.0,1.523816e+09,1.549698e+09,1.524961e+09,2.588216e+07,0.162485,11.182146
std,3.490933e+07,22.563242,18.635310,17.386150,6.267413,25.077151,24.844145,23.617415,3.240213,0.0,4.026490e+06,1.127524e+07,4.120079e+06,1.219490e+07,0.368896,4.186890
min,2.398315e+06,26.000000,11.000000,5.000000,1.000000,31.000000,17.000000,0.000000,-4.400000,0.0,1.517227e+09,1.517236e+09,1.517236e+09,1.000000e+00,0.000000,0.000000
25%,2.406520e+06,88.000000,62.000000,45.000000,91.000000,106.000000,106.000000,18.000000,36.300000,0.0,1.520000e+09,1.550941e+09,1.521107e+09,2.459543e+07,0.000000,13.000000
50%,7.049841e+07,103.000000,73.000000,55.000000,96.000000,120.000000,119.000000,25.000000,36.700000,0.0,1.524792e+09,1.553623e+09,1.525878e+09,2.958827e+07,0.000000,13.000000
75%,7.301918e+07,118.000000,87.000000,67.000000,99.000000,137.000000,137.000000,37.000000,37.200000,0.0,1.527338e+09,1.556309e+09,1.528707e+09,3.368889e+07,0.000000,13.000000
max,7.534191e+07,227.000000,179.000000,165.000000,100.000000,239.000000,296.000000,155.000000,40.500000,0.0,1.530155e+09,1.559000e+09,1.530090e+09,4.175980e+07,1.000000,13.000000


In [40]:
x3[((x3["ID"] ==2398315) & (x3["delta"] ==7202))]

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,fever,Date_timestamp,fever_date,fever_time,febrile_time,delta,metCriteria,sample_id,hourAway,nFeverEventID
417653,2398315,2018-01-31 22:21:32,78.0,57.0,47.0,95.0,126.0,126.0,53.0,37.2,0,1.517437e+09,2018-02-01 00:21:34,1.517444e+09,1.517444e+09,7202.0,1,2398315_3,2,2398315_1517444494.0
418554,2398315,2018-02-01 05:35:41,77.0,62.0,52.0,92.0,130.0,130.0,45.0,36.5,0,1.517463e+09,2018-02-01 07:35:43,1.517471e+09,1.517471e+09,7202.0,1,2398315_3,2,2398315_1517470543.0
420831,2398315,2018-02-01 23:42:07,88.0,66.0,56.0,95.0,107.0,93.0,30.0,36.1,0,1.517529e+09,2018-02-02 01:42:09,1.517536e+09,1.517536e+09,7202.0,1,2398315_3,2,2398315_1517535729.0
422186,2398315,2018-02-02 10:17:17,99.0,67.0,50.0,91.0,128.0,127.0,40.0,36.6,0,1.517567e+09,2018-02-02 12:17:19,1.517574e+09,1.517574e+09,7202.0,1,2398315_3,2,2398315_1517573839.0


In [54]:
#do only if error

x3 = x3.fillna(0)
#x3 = x3.drop(["fever_time_datetime"], axis = 1)
#x3.to_csv("x3.csv")

In [44]:
#changing nFeverTime to str
x3['nFeverEventID'] = x3['nFeverEventID'].astype('float64')
#x3 = x3.astype({"delta" : int})

ValueError: invalid literal for float(): 75341908_nan

In [42]:
x3.dtypes


ID                                  int64
Date                       datetime64[ns]
MDC_PRESS_CUFF_SYS                float64
MDC_PRESS_CUFF_MEAN               float64
MDC_PRESS_CUFF_DIA                float64
MDC_PULS_OXIM_SAT_O2              float64
MDC_PULS_OXIM_PULS_RATE           float64
MDC_ECG_CARD_BEAT_RATE            float64
MDC_TTHOR_RESP_RATE               float64
MDC_TEMP                          float64
fever                               int64
Date_timestamp                    float64
fever_date                 datetime64[ns]
fever_time                        float64
febrile_time                      float64
delta                             float64
metCriteria                         int64
sample_id                          object
hourAway                            int64
nFeverEventID                      object
dtype: object

In [2]:
x3 = pd.read_csv("x3.csv")

/sw/acf/anaconda2/4.4.0/centos7.3_gnu6.3.0/anaconda2-4.4.0/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
#x4.contains(str, regex=False)

In [3]:
#change feverTime to string
#searching string columns
#only take the variable, delta, nFeverEventID
x4 = x3[['MDC_PRESS_CUFF_SYS','MDC_PRESS_CUFF_MEAN','MDC_PRESS_CUFF_DIA','MDC_PULS_OXIM_SAT_O2','MDC_PULS_OXIM_PULS_RATE','MDC_ECG_CARD_BEAT_RATE','MDC_TTHOR_RESP_RATE','MDC_TEMP','nFeverEventID', 'delta']]
#x4 = x3[['nFeverEventID', 'delta', 'MDC_TEMP']]
#x4 = x4.iloc[1:50000]
#subset_columns = ['ID','Date','MDC_PRESS_CUFF_SYS','MDC_PRESS_CUFF_MEAN','MDC_PRESS_CUFF_DIA','MDC_PULS_OXIM_SAT_O2','MDC_PULS_OXIM_PULS_RATE','MDC_ECG_CARD_BEAT_RATE','MDC_TTHOR_RESP_RATE', 'MDC_TEMP']

In [ ]:
#importing tsfresh
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters

minimalFeatures = MinimalFCParameters()

#extract_features

#startHour = np.arange(0,1) ## Hours before alert time
#endObservation = 1 ## Observational Window length of each patient in hours
#lenStart = len(startHour)
#for i in range(0,lenStart):
    #works untill 30k samples
#check each rows with string
extracted_features = extract_features(x4, column_id = "nFeverEventID", column_sort = "delta", kind_to_fc_parameters=minimalFeatures)
#extracted_features.to_csv("Fever_extracted_features_t1.csv")

Feature Extraction:   0%|          | 0/79 [00:00<?, ?it/s]

In [ ]:
extracted_features.to_csv("Fever_minimal_features_all_columns.csv", index=False)

In [ ]:
testDa = {'col1': [1, 2], 'col2': ['a', 4]}
testdf = pd.DataFrame(data=testDa)

In [49]:
  ###################################################################################
###################older_method_below####################################################

In [56]:
dfxx2 = dfsub2[['ID', 'Date', 'fever_date', 'Date_timestamp', 'febrile_time', 'delta', 'metCriteria', 'MDC_TEMP', 'sample_id']]


In [60]:
dfxx2[((dfxx2["ID"] ==2398315) & (dfxx2["delta"] ==7202))]

,ID,Date,fever_date,Date_timestamp,febrile_time,delta,metCriteria,MDC_TEMP,sample_id
417652,2398315,2018-01-31 22:20:49,2018-02-01 00:20:51,1.517437e+09,1.517444e+09,7202.0,1,37.2,2398315_3
418553,2398315,2018-02-01 05:34:57,2018-02-01 07:34:59,1.517463e+09,1.517470e+09,7202.0,1,36.5,2398315_3
419649,2398315,2018-02-01 14:17:05,2018-02-01 16:17:07,1.517495e+09,1.517502e+09,7202.0,1,37.3,2398315_3
420829,2398315,2018-02-01 23:41:16,2018-02-02 01:41:18,1.517528e+09,1.517536e+09,7202.0,1,36.1,2398315_3
436374,2398315,2018-02-09 22:04:18,2018-02-10 00:04:20,1.518214e+09,1.518221e+09,7202.0,1,37.4,2398315_3


In [31]:
#droppping nas and converitng delta float into int

dfsub4 = dfsub3.dropna()
dfsub5 = dfsub4.astype({"delta" : int})
dfsub5
# dfsub5= pd.DataFrame(columns=['delta'])
# dfsub4.delta = dfsub4.delta.astype(int)
# dfsub4
#dfsub4

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,metCriteria,sample_id
412073,2398315,2018-01-30 00:13:47,148.0,112.0,93.0,99.0,106.0,105.0,42.0,39.5,3567,1,2398315_1
412074,2398315,2018-01-30 00:14:02,148.0,112.0,93.0,97.0,131.0,105.0,42.0,39.5,3552,1,2398315_1
412075,2398315,2018-01-30 00:14:47,148.0,112.0,93.0,99.0,113.0,109.0,53.0,39.5,3507,1,2398315_1
412076,2398315,2018-01-30 00:15:02,148.0,112.0,93.0,97.0,129.0,109.0,53.0,39.5,3492,1,2398315_1
412077,2398315,2018-01-30 00:15:47,148.0,112.0,93.0,100.0,103.0,103.0,67.0,39.5,3447,1,2398315_1
412078,2398315,2018-01-30 00:16:02,148.0,112.0,93.0,97.0,127.0,126.0,67.0,39.5,3432,1,2398315_1
412079,2398315,2018-01-30 00:16:47,148.0,112.0,93.0,100.0,101.0,116.0,67.0,39.5,3387,1,2398315_1
412080,2398315,2018-01-30 00:17:02,148.0,112.0,93.0,98.0,129.0,127.0,31.0,39.5,3372,1,2398315_1
412081,2398315,2018-01-30 00:17:47,148.0,112.0,93.0,100.0,97.0,94.0,34.0,39.5,3327,1,2398315_1
412082,2398315,2018-01-30 00:18:02,148.0,112.0,93.0,94.0,122.0,121.0,28.0,39.5,3312,1,2398315_1


In [21]:
#dfsub5.to_csv("tempStudy_clean.csv", index=False)


In [82]:
dfsub5 = pd.read_csv("tempStudy_clean.csv")

In [356]:
dfsub5

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,metCriteria,sample_id
0,2398315,2018-01-29 19:13:42,158.0,115.0,98.0,100.0,89.0,105.0,24.0,39.5,21572,1,2398315_6
1,2398315,2018-01-29 19:13:47,158.0,115.0,98.0,90.0,106.0,105.0,24.0,39.5,21567,1,2398315_6
2,2398315,2018-01-29 19:14:42,158.0,115.0,98.0,100.0,78.0,80.0,24.0,39.5,21512,1,2398315_6
3,2398315,2018-01-29 19:14:47,158.0,115.0,98.0,90.0,106.0,106.0,24.0,39.5,21507,1,2398315_6
4,2398315,2018-01-29 19:15:42,158.0,115.0,98.0,100.0,79.0,78.0,26.0,39.5,21452,1,2398315_6
5,2398315,2018-01-29 19:15:47,158.0,115.0,98.0,90.0,107.0,107.0,25.0,39.5,21447,1,2398315_6
6,2398315,2018-01-29 19:16:42,158.0,115.0,98.0,100.0,77.0,77.0,50.0,39.5,21392,1,2398315_6
7,2398315,2018-01-29 19:16:47,158.0,115.0,98.0,90.0,108.0,108.0,26.0,39.5,21387,1,2398315_6
8,2398315,2018-01-29 19:17:42,158.0,115.0,98.0,100.0,68.0,68.0,31.0,39.5,21332,1,2398315_6
9,2398315,2018-01-29 19:17:47,158.0,115.0,98.0,90.0,108.0,108.0,24.0,39.5,21327,1,2398315_6


In [357]:
#Drop columns ID, Date, metCriteria 
#if error try adding two two column to see which one gives error

#dfsub6 = dfsub5[['delta','sample_id','Date']]

dfsub6 = dfsub5.drop(['ID', 'Date', 'metCriteria'], axis = 1)
#del dfsub5
#dfsub6

#sorting by sample_id and delta
#df6_sort = dfsub6.sort_values(['sample_id'],['delta'])
#df6_sort
dfsub7 = dfsub6.sort_values(['sample_id', 'delta'], ascending=[True, True])
dfsub7

,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,sample_id
7931,115.0,85.0,65.0,95.0,132.0,132.0,15.0,37.9,50000,2398315_0
7932,115.0,85.0,65.0,94.0,91.0,91.0,23.0,37.9,50000,2398315_0
7933,115.0,85.0,65.0,95.0,132.0,131.0,15.0,37.9,50000,2398315_0
7934,115.0,85.0,65.0,95.0,94.0,95.0,22.0,37.9,50000,2398315_0
7935,115.0,85.0,65.0,95.0,132.0,131.0,15.0,37.9,50000,2398315_0
7936,115.0,85.0,65.0,95.0,89.0,88.0,15.0,37.9,50000,2398315_0
7937,115.0,85.0,65.0,95.0,89.0,88.0,19.0,37.9,50000,2398315_0
7938,115.0,85.0,65.0,94.0,132.0,131.0,15.0,37.9,50000,2398315_0
7939,115.0,85.0,65.0,93.0,95.0,95.0,24.0,37.9,50000,2398315_0
7940,115.0,85.0,65.0,95.0,132.0,131.0,15.0,37.9,50000,2398315_0


In [413]:
dfsub5[((dfsub5["sample_id"] =='2398315_3') & (dfsub5["delta"] ==7202))]

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,metCriteria,sample_id
3122,2398315,2018-01-31 22:21:32,81.0,56.0,42.0,95.0,126.0,126.0,53.0,37.2,7202,1,2398315_3
3875,2398315,2018-02-01 05:35:41,89.0,73.0,64.0,92.0,130.0,130.0,45.0,36.5,7202,1,2398315_3
5162,2398315,2018-02-01 23:42:07,113.0,76.0,55.0,96.0,107.0,107.0,30.0,36.1,7202,1,2398315_3
5923,2398315,2018-02-02 10:17:17,108.0,70.0,51.0,91.0,128.0,127.0,40.0,36.6,7202,1,2398315_3


In [358]:
#dfsub7[(dfsub7['sample_id']== '75341908_6')]

#Create a nwe delta and make a func to subtract delta based sample_id
#delta should start from 0 and go up
#dfsub8 = dfsub7
#dfsub8
#dfsub8.drop(['dfsub8', 'deltaNew'], axis = 1)
#df['dA'] = df['A'] - df['A'].shift(-1)
# dfsub8["deltaNew"] = dfsub7["delta"].apply(lambda x: x[i] - x[i-1])
# dfsub8


# dfsub8['deltaReduced'] = [i - dfsub8['delta'][0] for i in dfsub8['delta']]
# dfsub8
#

#dfsub8[dfsub8["sample_id"].str.contains("_{0}".format(6)) == True]

#ofor _

# lst = []
# for i in range(0,7):
#     temp_df = dfsub8[dfsub8["sample_id"].str.contains("_{0}".format(i)) == True]
#     lt = list(temp_df['delta'])
#     lst.extend([x - lt[0] for x in lt])
    
# dfsub8['delta_Reduced'] = lst
# dfsub8

lst = []
for i in dfsub7["sample_id"].unique():
    temp_df = dfsub7[dfsub7["sample_id"] == '{0}'.format(i)]
    lt = list(temp_df['delta'])
    lst.extend([x - lt[0] for x in lt])
    
dfsub7['delta_Reduced'] = lst
dfsub7

#dfsub9 = dfsub8.drop(['deltaReduced'], axis = 1)
dfsub9 = dfsub7

In [359]:
dfsub9.tail(500)

,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,sample_id,delta_Reduced
1400394,93.0,73.0,63.0,98.0,98.0,98.0,22.0,36.8,50000,75341908_0,0
1400395,93.0,73.0,63.0,97.0,120.0,120.0,55.0,36.8,50000,75341908_0,0
1400396,93.0,73.0,63.0,98.0,90.0,90.0,23.0,36.8,50000,75341908_0,0
1400397,93.0,73.0,63.0,97.0,98.0,98.0,24.0,36.8,50000,75341908_0,0
1400398,93.0,73.0,63.0,98.0,103.0,103.0,25.0,36.8,50000,75341908_0,0
1400399,93.0,73.0,63.0,97.0,103.0,103.0,24.0,36.8,50000,75341908_0,0
1400400,93.0,73.0,63.0,97.0,105.0,105.0,24.0,36.8,50000,75341908_0,0
1400401,93.0,73.0,63.0,97.0,97.0,97.0,23.0,36.8,50000,75341908_0,0
1400402,93.0,73.0,63.0,97.0,96.0,96.0,23.0,36.8,50000,75341908_0,0
1400403,93.0,73.0,63.0,97.0,92.0,92.0,23.0,36.8,50000,75341908_0,0


In [360]:
dfsub9[dfsub9["sample_id"] =='2398315_3']

,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,sample_id,delta_Reduced
3122,81.0,56.0,42.0,95.0,126.0,126.0,53.0,37.2,7202,2398315_3,0
3875,89.0,73.0,64.0,92.0,130.0,130.0,45.0,36.5,7202,2398315_3,0
5162,113.0,76.0,55.0,96.0,107.0,107.0,30.0,36.1,7202,2398315_3,0
5923,108.0,70.0,51.0,91.0,128.0,127.0,40.0,36.6,7202,2398315_3,0
4398,112.0,70.0,53.0,96.0,124.0,124.0,26.0,37.3,7203,2398315_3,1
5161,113.0,76.0,55.0,96.0,95.0,107.0,20.0,36.1,7203,2398315_3,1
7647,116.0,87.0,68.0,92.0,129.0,129.0,15.0,37.4,7204,2398315_3,2
7646,116.0,87.0,68.0,84.0,121.0,123.0,30.0,37.4,7205,2398315_3,3
7645,116.0,87.0,68.0,84.0,121.0,123.0,30.0,37.4,7206,2398315_3,4
1821,92.0,63.0,46.0,99.0,98.0,97.0,23.0,35.9,7207,2398315_3,5


In [361]:
# use assignement to a new data frame for testing purposes
# dfsub9["delta_Reduced"].replace(0, np.nan, inplace=True)
# dfsub9

In [362]:
dfsub9.nunique()

MDC_PRESS_CUFF_SYS           166
MDC_PRESS_CUFF_MEAN          154
MDC_PRESS_CUFF_DIA           141
MDC_PULS_OXIM_SAT_O2          94
MDC_PULS_OXIM_PULS_RATE      208
MDC_ECG_CARD_BEAT_RATE       264
MDC_TTHOR_RESP_RATE          147
MDC_TEMP                     333
delta                      20285
sample_id                    514
delta_Reduced               3600
dtype: int64

In [363]:
#  dfsub8.groupby(['sample_id'])['delta'].agg(lambda x: (x.iloc[-1] - x.iloc[0]))
#dfsub9 = dfsub9.loc[0:30000]
#dfsub9

In [364]:
dfsub9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1400444 entries, 7931 to 1399874
Data columns (total 11 columns):
MDC_PRESS_CUFF_SYS         1400444 non-null float64
MDC_PRESS_CUFF_MEAN        1400444 non-null float64
MDC_PRESS_CUFF_DIA         1400444 non-null float64
MDC_PULS_OXIM_SAT_O2       1400444 non-null float64
MDC_PULS_OXIM_PULS_RATE    1400444 non-null float64
MDC_ECG_CARD_BEAT_RATE     1400444 non-null float64
MDC_TTHOR_RESP_RATE        1400444 non-null float64
MDC_TEMP                   1400444 non-null float64
delta                      1400444 non-null int64
sample_id                  1400444 non-null object
delta_Reduced              1400444 non-null int64
dtypes: float64(8), int64(2), object(1)
memory usage: 128.2+ MB


In [365]:
dfsub9_t1 = dfsub9[dfsub9['sample_id'].str.contains("_1")]

In [366]:
dfsub9_t2 = dfsub9[dfsub9['sample_id'].str.contains("_2")]

In [367]:
dfsub9_t3 = dfsub9[dfsub9['sample_id'].str.contains("_3")]

In [368]:
dfsub9_t4 = dfsub9[dfsub9['sample_id'].str.contains("_4")]

In [369]:
dfsub9_t5 = dfsub9[dfsub9['sample_id'].str.contains("_5")]

In [370]:
dfsub9_t6 = dfsub9[dfsub9['sample_id'].str.contains("_6")]

In [371]:
dfsub9_t0 = dfsub9[dfsub9['sample_id'].str.contains("_0")]
#dfsub7 = dfsub6.loc[0:30000]
#dfsub6_t0 = dfsub7[dfsub7['sample_id'].str.contains("_0")] #dfsub7 = dfsub6.loc[0:30000]

In [372]:
dfsub9_t0

,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,sample_id,delta_Reduced
7931,115.0,85.0,65.0,95.0,132.0,132.0,15.0,37.9,50000,2398315_0,0
7932,115.0,85.0,65.0,94.0,91.0,91.0,23.0,37.9,50000,2398315_0,0
7933,115.0,85.0,65.0,95.0,132.0,131.0,15.0,37.9,50000,2398315_0,0
7934,115.0,85.0,65.0,95.0,94.0,95.0,22.0,37.9,50000,2398315_0,0
7935,115.0,85.0,65.0,95.0,132.0,131.0,15.0,37.9,50000,2398315_0,0
7936,115.0,85.0,65.0,95.0,89.0,88.0,15.0,37.9,50000,2398315_0,0
7937,115.0,85.0,65.0,95.0,89.0,88.0,19.0,37.9,50000,2398315_0,0
7938,115.0,85.0,65.0,94.0,132.0,131.0,15.0,37.9,50000,2398315_0,0
7939,115.0,85.0,65.0,93.0,95.0,95.0,24.0,37.9,50000,2398315_0,0
7940,115.0,85.0,65.0,95.0,132.0,131.0,15.0,37.9,50000,2398315_0,0


In [48]:
#imporintg tsfresh
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

# extract_features
# make a counter

#startHour = np.arange(0,1) ## Hours before alert time
#endObservation = 1 ## Observational Window length of each patient in hours
#lenStart = len(startHour)
#for i in range(0,lenStart):
    #works untill 30k samples



ImportError: No module named tsfresh

In [374]:
#imporintg tsfresh
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

# extract_features
# make a counter

#startHour = np.arange(0,1) ## Hours before alert time
#endObservation = 1 ## Observational Window length of each patient in hours
#lenStart = len(startHour)
#for i in range(0,lenStart):
    #works untill 30k samples


extracted_features = extract_features(dfsub9_t1, column_id = "sample_id", column_sort = "delta_Reduced")
extracted_features.to_csv("Fever_extracted_features_t1.csv")






Feature Extraction:   0%|                                                                       | 0/98 [00:00<?, ?it/s]




Feature Extraction:   1%|▋                                                              | 1/98 [00:06<11:11,  6.92s/it]




Feature Extraction:   2%|█▎                                                             | 2/98 [00:07<06:02,  3.77s/it]




Feature Extraction:   3%|█▉                                                             | 3/98 [00:07<04:05,  2.59s/it]




Feature Extraction:   4%|██▌                                                            | 4/98 [00:08<03:28,  2.21s/it]




Feature Extraction:   7%|████▌                                                          | 7/98 [00:09<02:00,  1.32s/it]




Feature Extraction:   8%|█████▏                                                         | 8/98 [00:14<02:38,  1.76s/it]




Feature Extraction:  10%|██████▎                                                       | 10/98 [00:18<02:43,  1.85s/it]




Fea

In [160]:
extracted_features = extract_features(dfsub9_t2, column_id = "sample_id", column_sort = "delta_Reduced")
extracted_features.to_csv("Fever_extracted_features_t2.csv")

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 92/92 [05:02<00:00,  3.28s/it]


In [385]:
extracted_features = extract_features(dfsub9_t3, column_id = "sample_id", column_sort = "delta_Reduced")
extracted_features.to_csv("Fever_extracted_features_t3.csv")








Feature Extraction:   0%|                                                                       | 0/89 [00:00<?, ?it/s]






Feature Extraction:   1%|▋                                                              | 1/89 [00:27<41:01, 27.97s/it]






Feature Extraction:   2%|█▍                                                             | 2/89 [00:29<21:31, 14.84s/it]






Feature Extraction:   3%|██                                                             | 3/89 [00:30<14:20, 10.00s/it]






Feature Extraction:   4%|██▊                                                            | 4/89 [00:31<11:13,  7.92s/it]






Feature Extraction:   6%|███▌                                                           | 5/89 [00:46<12:56,  9.24s/it]






Feature Extraction:   7%|████▏                                                          | 6/89 [00:46<10:49,  7.83s/it]






Feature Extraction:   8%|████▉                                                          | 7/89 [01:19<15:26, 11

Feature Extraction:  74%|█████████████████████████████████████████████▉                | 66/89 [09:53<03:26,  8.99s/it]






Feature Extraction:  75%|██████████████████████████████████████████████▋               | 67/89 [10:35<03:28,  9.48s/it]






Feature Extraction:  76%|███████████████████████████████████████████████▎              | 68/89 [10:35<03:16,  9.35s/it]






Feature Extraction:  78%|████████████████████████████████████████████████              | 69/89 [10:43<03:06,  9.32s/it]






Feature Extraction:  79%|████████████████████████████████████████████████▊             | 70/89 [10:54<02:57,  9.34s/it]






Feature Extraction:  80%|█████████████████████████████████████████████████▍            | 71/89 [10:56<02:46,  9.24s/it]






Feature Extraction:  81%|██████████████████████████████████████████████████▏           | 72/89 [11:02<02:36,  9.20s/it]






Feature Extraction:  82%|██████████████████████████████████████████████████▊           | 73/89 [11:15<02:28,  9.26s/it

In [31]:
extracted_features = extract_features(dfsub9_t4, column_id = "sample_id", column_sort = "delta_Reduced")
extracted_features.to_csv("Fever_extracted_features_t4.csv")

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 90/90 [04:38<00:00,  3.10s/it]


In [32]:
extracted_features = extract_features(dfsub9_t5, column_id = "sample_id", column_sort = "delta_Reduced")
extracted_features.to_csv("Fever_extracted_features_t5.csv")

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 84/84 [04:26<00:00,  3.17s/it]


In [ ]:
extracted_features = extract_features(dfsub9_t6, column_id = "sample_id", column_sort = "delta_Reduced")
extracted_features.to_csv("Fever_extracted_features_t6.csv")

In [384]:
#t0 has more than 1.2 mil rows. Memry error pops when the data >20k
extracted_features = extract_features(dfsub9_t0, column_id = "sample_id", column_sort = "delta_Reduced")
#extracted_features.to_csv("Fever_extracted_features_t0.csv")







Feature Extraction:   0%|                                                                       | 0/99 [00:00<?, ?it/s]

MemoryError: 

In [190]:
dfsub9_t2

,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,sample_id,delta_Reduced
3244,115.0,74.0,58.0,95.0,148.0,147.0,60.0,37.6,3601,2398315_2,0
4005,112.0,67.0,52.0,92.0,143.0,142.0,60.0,37.7,3601,2398315_2,0
5287,119.0,65.0,43.0,90.0,126.0,126.0,43.0,37.3,3601,2398315_2,0
4525,106.0,63.0,47.0,97.0,130.0,129.0,56.0,37.7,3602,2398315_2,0
7779,118.0,86.0,67.0,93.0,130.0,129.0,15.0,37.6,3602,2398315_2,0
6948,80.0,31.0,55.0,90.0,127.0,126.0,52.0,37.6,3605,2398315_2,0
7778,118.0,86.0,67.0,94.0,122.0,122.0,23.0,37.6,3605,2398315_2,0
1964,90.0,61.0,42.0,96.0,131.0,131.0,21.0,37.1,3606,2398315_2,0
2579,112.0,71.0,54.0,99.0,136.0,135.0,49.0,37.5,3610,2398315_2,0
602,148.0,112.0,93.0,95.0,133.0,100.0,44.0,39.5,3612,2398315_2,0


In [187]:
dfsub9_t0[(dfsub9_t0["sample_id"]=='75341908_0')]

,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,sample_id,delta_Reduced
1400240,103.0,84.0,74.0,98.0,115.0,115.0,27.0,37.9,50000,75341908_0,1873
1400241,103.0,84.0,74.0,100.0,115.0,115.0,25.0,37.9,50000,75341908_0,1903
1400242,103.0,84.0,74.0,100.0,108.0,108.0,24.0,37.9,50000,75341908_0,1904
1400243,103.0,84.0,74.0,100.0,102.0,102.0,24.0,37.9,50000,75341908_0,1934
1400244,103.0,84.0,74.0,100.0,107.0,107.0,25.0,37.9,50000,75341908_0,1935
1400245,103.0,84.0,74.0,100.0,106.0,106.0,24.0,37.9,50000,75341908_0,1965
1400246,103.0,84.0,74.0,100.0,106.0,106.0,24.0,37.9,50000,75341908_0,1966
1400247,103.0,84.0,74.0,100.0,102.0,102.0,25.0,37.9,50000,75341908_0,1996
1400248,103.0,84.0,74.0,100.0,107.0,107.0,25.0,37.8,50000,75341908_0,1997
1400249,103.0,84.0,74.0,100.0,111.0,111.0,25.0,37.8,50000,75341908_0,2023


In [188]:
dfsub8[(dfsub8["sample_id"]=='75341908_0')]

,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,sample_id,deltaReduced,delta_Reduced
1400240,103.0,84.0,74.0,98.0,115.0,115.0,27.0,37.9,50000,75341908_0,71572,1873
1400241,103.0,84.0,74.0,100.0,115.0,115.0,25.0,37.9,50000,75341908_0,71572,1903
1400242,103.0,84.0,74.0,100.0,108.0,108.0,24.0,37.9,50000,75341908_0,71572,1904
1400243,103.0,84.0,74.0,100.0,102.0,102.0,24.0,37.9,50000,75341908_0,71572,1934
1400244,103.0,84.0,74.0,100.0,107.0,107.0,25.0,37.9,50000,75341908_0,71572,1935
1400245,103.0,84.0,74.0,100.0,106.0,106.0,24.0,37.9,50000,75341908_0,71572,1965
1400246,103.0,84.0,74.0,100.0,106.0,106.0,24.0,37.9,50000,75341908_0,71572,1966
1400247,103.0,84.0,74.0,100.0,102.0,102.0,25.0,37.9,50000,75341908_0,71572,1996
1400248,103.0,84.0,74.0,100.0,107.0,107.0,25.0,37.8,50000,75341908_0,71572,1997
1400249,103.0,84.0,74.0,100.0,111.0,111.0,25.0,37.8,50000,75341908_0,71572,2023


In [58]:
from pandas import *

# tp = read_csv('large_dataset.csv', iterator=True, chunksize=1000)  # gives TextFileReader, which is iterable with chunks of 1000 rows.
# df = concat(tp, ignore_index=True) 
tp = read_csv('tempStudy_clean.csv', iterator = True, chunksize = 1000)
testdf = concat(tp, ignore_index = True)
testdf

,ID,Date,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,metCriteria,sample_id
0,2398315,2018-01-29 19:13:42,158.0,115.0,98.0,100.0,89.0,105.0,24.0,39.5,21572,1,2398315_6
1,2398315,2018-01-29 19:13:47,158.0,115.0,98.0,90.0,106.0,105.0,24.0,39.5,21567,1,2398315_6
2,2398315,2018-01-29 19:14:42,158.0,115.0,98.0,100.0,78.0,80.0,24.0,39.5,21512,1,2398315_6
3,2398315,2018-01-29 19:14:47,158.0,115.0,98.0,90.0,106.0,106.0,24.0,39.5,21507,1,2398315_6
4,2398315,2018-01-29 19:15:42,158.0,115.0,98.0,100.0,79.0,78.0,26.0,39.5,21452,1,2398315_6
5,2398315,2018-01-29 19:15:47,158.0,115.0,98.0,90.0,107.0,107.0,25.0,39.5,21447,1,2398315_6
6,2398315,2018-01-29 19:16:42,158.0,115.0,98.0,100.0,77.0,77.0,50.0,39.5,21392,1,2398315_6
7,2398315,2018-01-29 19:16:47,158.0,115.0,98.0,90.0,108.0,108.0,26.0,39.5,21387,1,2398315_6
8,2398315,2018-01-29 19:17:42,158.0,115.0,98.0,100.0,68.0,68.0,31.0,39.5,21332,1,2398315_6
9,2398315,2018-01-29 19:17:47,158.0,115.0,98.0,90.0,108.0,108.0,24.0,39.5,21327,1,2398315_6


In [59]:
testdf1 = testdf.drop(['ID', 'Date', 'metCriteria'], axis = 1)
testdf1

,MDC_PRESS_CUFF_SYS,MDC_PRESS_CUFF_MEAN,MDC_PRESS_CUFF_DIA,MDC_PULS_OXIM_SAT_O2,MDC_PULS_OXIM_PULS_RATE,MDC_ECG_CARD_BEAT_RATE,MDC_TTHOR_RESP_RATE,MDC_TEMP,delta,sample_id
0,158.0,115.0,98.0,100.0,89.0,105.0,24.0,39.5,21572,2398315_6
1,158.0,115.0,98.0,90.0,106.0,105.0,24.0,39.5,21567,2398315_6
2,158.0,115.0,98.0,100.0,78.0,80.0,24.0,39.5,21512,2398315_6
3,158.0,115.0,98.0,90.0,106.0,106.0,24.0,39.5,21507,2398315_6
4,158.0,115.0,98.0,100.0,79.0,78.0,26.0,39.5,21452,2398315_6
5,158.0,115.0,98.0,90.0,107.0,107.0,25.0,39.5,21447,2398315_6
6,158.0,115.0,98.0,100.0,77.0,77.0,50.0,39.5,21392,2398315_6
7,158.0,115.0,98.0,90.0,108.0,108.0,26.0,39.5,21387,2398315_6
8,158.0,115.0,98.0,100.0,68.0,68.0,31.0,39.5,21332,2398315_6
9,158.0,115.0,98.0,90.0,108.0,108.0,24.0,39.5,21327,2398315_6


In [60]:
extracted_features = extract_features(testdf1, column_id = "sample_id", column_sort = "delta")




Feature Extraction:   0%|                                                                       | 0/98 [00:00<?, ?it/s]


Feature Extraction:   1%|▌                                                          | 1/98 [13:09<21:15:43, 789.11s/it]

MemoryError: 